Conectado a Python 3.12.8

In [2]:
import os
from time import time as t
import folium
import geopandas
import numpy as np
from multiprocessing import Pool
from shapely.geometry import LineString,MultiLineString,shape,Point
from scipy.spatial import Voronoi
#import topojson
import argparse
from pyInegi.shapely_tools import intersection_points

In [3]:
def renombrar(name):
	noms = name.split(".")
	if not os.path.exists(name): 
		return name
	newName = (noms[0][:-1] + str(int(noms[0][-1]) + 1) if noms[0][-1].isdigit() else f"{noms[0]}_1")
	nomComp = f"{newName}.{noms[1]}"
	return renombrar(nomComp)

In [14]:
def inicio(a):
	ini=t()
	orig =  geopandas.read_file(a.file,rows=a.rows)
	orig.set_index("OBJECTID",inplace=True)
	CRS = orig.crs.to_string()
	segm = orig.segmentize(a.dist)
	voroPoly = segm.voronoi_polygons()
	borde = segm.boundary
	voroClip=voroPoly.boundary.clip(orig)
	union = voroClip.union_all()
	linSep=list(union.geoms)
	df = geopandas.GeoDataFrame(geometry=linSep,crs=CRS)
	inter,ids=intersection_points(linSep,list(borde.iloc[0].geoms),0.1)
	df=df.drop(index=ids)
	Central = df.union_all("unary")
	df_union = geopandas.GeoDataFrame(geometry=[Central],crs=CRS)
	borde.to_file(renombrar("DatosSalida/borde.shp"))
	df_union.to_file(renombrar("DatosSalida/noRepeat.shp"))
	return [orig,segm,voroPoly,voroClip,union,Central]

In [6]:
class argumentos:
	def __init__(self,f,r,d):
		self.file=f
		self.rows=r
		self.dist=d

In [ ]:
args=argumentos("prueba3.shp",2,5)
inicio(args)

In [23]:
r = inicio(args)
print(r[4].explode())

AttributeError: 'MultiLineString' object has no attribute 'explode'

In [24]:
dir(r[4])

['__and__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 '_geom',
 '_geom_prepared',
 '_ndim',
 '_repr_svg_',
 'almost_equals',
 'area',
 'boundary',
 'bounds',
 'buffer',
 'centroid',
 'contains',
 'contains_properly',
 'convex_hull',
 'coords',
 'covered_by',
 'covers',
 'crosses',
 'difference',
 'disjoint',
 'distance',
 'dwithin',
 'envelope',
 'equals',
 'equals_exact',
 'geom_type',
 'geometryType',
 'geoms',
 'has_z',
 'hausdorff_distance',
 'interpolate',
 'intersection',
 'intersects',
 'is_closed',
 'is_empty',
 'is_ring',
 'is_simple',
 'is_valid',
 '

In [40]:
linea=[]
for x in list(r[4].geoms):
    linea=[Point(x.coords[0])]
    linea.append(Point(x.coords[1]))
    for c in list(x.coords):
        if Point(c) not in linea:
            linea.append(Point(c))
print(linea)

KeyboardInterrupt: 